# Transformers 模型量化技术：GPTQ

## 使用 GPTQ 量化模型
### 使用`GPTQ`算法支持的默认数据集来量化

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

mode_name_or_path = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    dataset="wikitext2",
    desc_act=False
)

In [3]:
# 逐层量化


quant_model = AutoModelForCausalLM.from_pretrained(
    mode_name_or_path,
    quantization_config=quantization_config,
    device_map='auto'
)

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

### 检查量化模型正确性

In [5]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[ 1766754698, -1249142373,  1183631034,  ..., -2038658921,
                        -2037544795, -1956877206],
                       [ 1772710025,  1739893370, -1500087466,  ...,  2021033895,
                         -662329995,  1756019066],
                       [ -895658394, -2007414633, -1951893913,  ..., -1429760649,
                         -980833883,  1451914633],
                       ...,
                       [ 2025363833, -1412855115,  1539086490,  ...,  -342189415,
                        -1737062521, -1950833303],
                       [ 2023326856, -1432974698, -1788251003,  ..., -1161734024,
                         2043176297,  1449571741],
                       [ 1769506966, -2021291654, -1182103157,  ...,  1999993942,
                         1753778600,  1970816134]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

In [6]:
# 保存模型权重

quant_model.save_pretrained("models/opt-2.7b-gptq")

#### 使用 GPU 加载模型并生成文本

In [7]:
tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

/home/ecs-user/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Merry Christmas! I'm glad to see you're still around.
I'm still around, just not posting as much.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

In [ ]:
## 这个不需要
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(mode_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
# week 4 作业-2

# HOMEWORK: 使用GPTQ算法量化OPT-6.7B模型
# 作业可选，需要GPU超过16G的显存

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

mode_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    dataset="wikitext2",
    desc_act=False
)

In [2]:
# 逐层量化

# 这个会量化模型

quant_model = AutoModelForCausalLM.from_pretrained(
    mode_name_or_path,
    quantization_config=quantization_config,
    device_map='auto'
)

/home/ecs-user/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
CUDA extension not installed.
CUDA extension not installed.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ecs-user/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 

In [ ]:
# 验证是否量化了
quant_model.decoder.layers[0].self_attn.q_proj.__dict__

In [ ]:
# 保存模型权重

quant_model.save_pretrained("models/opt-6.7b-gptq")

In [ ]:
# 使用训练的模型

tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:

# GPU内存用完了，没有完成6.7B的训练